### Load package and set path

In [5]:
import os,sys
import pygeos
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import matplotlib.pyplot as plt

from shapely.geometry import mapping
from pathlib import Path
from pgpkg import Geopackage
from tqdm import tqdm
from rasterstats import zonal_stats
#pip install geofeather
from geofeather.pygeos import to_geofeather, from_geofeather

sys.path.append("C:\Projects\Future_exposure\scripts")
import functions

In [6]:
#load pathways
#grid_data = gpd.read_file(os.path.join(os.path.abspath(os.path.join('C:/Users/snn490/surfdrive','Outputs','Grid_data')),'global_grid_1degree.gpkg')) #open as geofeather
#glob_info = pd.read_excel(os.path.join(r'C:\Users\snn490\surfdrive\Datasets','global_information.xlsx'))

shape_countries = gpd.read_file(os.path.join(os.path.abspath(r'C:\Users\snn490\surfdrive\Datasets\Administrative_boundaries\global_countries_buffer'),"global_countries_advanced.shp"))
pop_data_path = os.path.abspath(os.path.join('C:/Users/snn490/surfdrive','Datasets','Population_data')) #save interim calculations
pop_data_clip_path = os.path.abspath(r'\\labsdfs.labs.vu.nl\labsdfs\BETA-IVM-BAZIS\snn490\Datasets\Population_data\per_country')

C:\Users\snn490\surfdrive\Datasets\Population_data

#create lists
country_lst = ['FRA','DEU','ITA']
year_lst = ['2020', '2100']
ssp_lst = ['SSP1', 'SSP3', 'SSP5']

In [7]:
#create clipped population raster files https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/crop-raster-data-with-shapefile-in-python/
for ssp in ssp_lst:
    pop_data_2020 = rxr.open_rasterio(os.path.join(pop_data_path,'DIVA_{}_2020.tif'.format(ssp)),masked=True).squeeze()
    pop_data_2100 = rxr.open_rasterio(os.path.join(pop_data_path,'DIVA_{}_2100.tif'.format(ssp)),masked=True).squeeze()
    for country in country_lst:
        country_shape = shape_countries[shape_countries['ISO_3digit'] == country] #get row with coordinates corresponding to country code
        print(country_shape)
        
        # check coordinate systems, if crs the same then continue clipping
        print('crop extent crs: ', country_shape['geometry'].crs)
        print('lidar crs: ', pop_data_2020.rio.crs)
        
        pop_data_2020_clipped = pop_data_2020.rio.clip(country_shape.geometry.apply(mapping))
        pop_data_2100_clipped = pop_data_2100.rio.clip(country_shape.geometry.apply(mapping))

        # Write the data to a new geotiff file
        tif_file_path = os.path.abspath(os.path.join(pop_data_path, 'per_country')) 
        Path(tif_file_path).mkdir(parents=True, exist_ok=True) #create new directory, if it doesn't exist yet
        pop_data_2020_clipped.rio.to_raster(os.path.join(tif_file_path, '{}_{}_2020.tif'.format(country, ssp)))
        pop_data_2100_clipped.rio.to_raster(os.path.join(tif_file_path, '{}_{}_2100.tif'.format(country, ssp)))

        

     fid GID_0  NAME_0  ID_0 ISO_3digit Country  CC a-2  \
71  71.0   FRA  France  79.0        FRA  France  EU  FR   

                       Name continent  SIDS capital           coordinate  \
71  France, French Republic    Europe   0.0   Paris  (48.85661, 2.35222)   

   country_1 wbregion   wbregionna wbincome         wbincomena  \
71       FRA      YHI  High Income      HIO  High income: OECD   

                                             geometry  
71  MULTIPOLYGON (((8.54420 42.33779, 8.54287 42.3...  
crop extent crs:  epsg:4326
lidar crs:  EPSG:4326


MemoryError: Unable to allocate 6.95 GiB for an array with shape (21600, 43200) and data type float64

In [32]:
# subtract files https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/subtract-rasters-in-python/

for ssp in ssp_lst:
    for country in country_lst:
        # load data
        pop_data_2020 = rxr.open_rasterio(os.path.join(pop_data_clip_path,'{}_{}_2020.tif'.format(country, ssp)),masked=True).squeeze()
        pop_data_2100 = rxr.open_rasterio(os.path.join(pop_data_clip_path,'{}_{}_2100.tif'.format(country, ssp)),masked=True).squeeze()
        
        # double check bounds and resolution
        # Are the bounds the same?
        print("Is the spatial extent the same?",
              pop_data_2020.rio.bounds() == pop_data_2100.rio.bounds())

        # Is the resolution the same ??
        print("Is the resolution the same?",
              pop_data_2020.rio.resolution() == pop_data_2100.rio.resolution())
        
        # if both true, then it is safe to use easy method for calculating the change in population over time
        perc_change_pop = ((pop_data_2100-pop_data_2020)/pop_data_2020)*100
        
        # Write the data to a new geotiff file
        tif_file_path = os.path.abspath(os.path.join(pop_data_path, 'per_country_change')) 
        Path(tif_file_path).mkdir(parents=True, exist_ok=True) #create new directory, if it doesn't exist yet
        perc_change_pop.rio.to_raster(os.path.join(tif_file_path, '{}_{}_rel_change_2020_2100.tif'.format(country, ssp)))      

Is the spatial extent the same? True
Is the resolution the same? True
Is the spatial extent the same? True
Is the resolution the same? True
Is the spatial extent the same? True
Is the resolution the same? True
Is the spatial extent the same? True
Is the resolution the same? True
Is the spatial extent the same? True
Is the resolution the same? True
Is the spatial extent the same? True
Is the resolution the same? True
Is the spatial extent the same? True
Is the resolution the same? True
Is the spatial extent the same? True
Is the resolution the same? True
Is the spatial extent the same? True
Is the resolution the same? True


In [ ]:
# aggregate population at cisi-level, calculate changes, and project on CISI
def value_zonal_stat(x,world_grid):
    try:
        return zonal_stats(x,world_grid,stats="sum")
    except:
        return [{'sum': 0}]

#set pathways
cisi_path = os.path.abspath(r'\\labsdfs.labs.vu.nl\labsdfs\BETA-IVM-BAZIS\snn490\Outputs\Exposure\Final_cisi_files\CISI\010_degree')
country_shapes_path = os.path.abspath(r'C:\Users\snn490\surfdrive\Datasets\Administrative_boundaries\global_countries_buffer')

#import data
#cisi_index = pd.read_feather(cisi_path)
cisi_globe_pd = from_geofeather(os.path.join(cisi_path, 'CISI_global.feather')) 
shape_countries = from_geofeather(os.path.join(country_shapes_path,'global_countries_advanced.geofeather'))

for country in country_lst:
    # soft clip of cisi for countries
    country_shape = shape_countries[shape_countries['ISO_3digit'] == country]
    if country_shape.empty == False: #if ISO_3digit in shape_countries
        spat_tree = pygeos.STRtree(cisi_globe_pd.geometry)
        cisi_country = (cisi_globe_pd.loc[spat_tree.query(country_shape.geometry.iloc[0],predicate='intersects').tolist()]).sort_index(ascending=True) #get grids that overlap with cover_box
        cisi_country = cisi_country.reset_index(drop=True) #get index as column and name column grid_number

        #aggregate data
        for ssp in ssp_lst:
            # load data
            #pop_data_2020 = rxr.open_rasterio(os.path.join(pop_data_clip_path,'{}_{}_2020.tif'.format(country, ssp)),masked=True).squeeze()
            #pop_data_2100 = rxr.open_rasterio(os.path.join(pop_data_clip_path,'{}_{}_2100.tif'.format(country, ssp)),masked=True).squeeze()

            pop_data_2020 = os.path.join(pop_data_clip_path,'{}_{}_2020.tif'.format(country, ssp))
            pop_data_2100 = os.path.join(pop_data_clip_path,'{}_{}_2100.tif'.format(country, ssp))

            temp_df = functions.transform_to_gpd((cisi_country)) #transform df to gpd with shapely geometries

            #calculate tot_pop_2020 per CISI grid cell
            print('aggregate population data for {} scenario {}'.format(country, ssp))
            tqdm.pandas(desc='pop')
            temp_df['tot_pop_2020'] = temp_df.geometry.progress_apply(lambda x: value_zonal_stat(x,pop_data_2020))
            temp_df['tot_pop_2020'] = temp_df['tot_pop_2020'].apply(lambda x: x[0]['sum']) 

            #calculate tot_pop_2100 per CISI grid cell
            tqdm.pandas(desc='pop')
            temp_df['tot_pop_2100'] = temp_df.geometry.progress_apply(lambda x: value_zonal_stat(x,pop_data_2100))
            temp_df['tot_pop_2100'] = temp_df['tot_pop_2100'].apply(lambda x: x[0]['sum']) 

            temp_df['rel_change_pop'] = (temp_df["tot_pop_2100"] - temp_df["tot_pop_2020"])/ temp_df["tot_pop_2020"] * 100 #calculate change population
            temp_df['CISI_2100'] = temp_df['CISI'] + (temp_df['CISI']*(temp_df['rel_change_pop']/100)) #calculate change CISI

            # create new path
            aggr_change_path = os.path.abspath(os.path.join(r'\\labsdfs.labs.vu.nl\labsdfs\BETA-IVM-BAZIS\snn490\Datasets\Population_data', 'per_country_aggregated_change')) 
            Path(aggr_change_path).mkdir(parents=True, exist_ok=True) #create new directory, if it doesn't exist yet

            #save as gpkg
            temp_df.to_file(os.path.join(aggr_change_path , '{}_{}_rel_change_2020_2100.gpkg'.format(country, ssp)), layer=' ', driver="GPKG")
            #save as geofeather
            temp_wkb = pd.DataFrame(temp_df) 
            temp_wkb['geometry'] = pygeos.from_shapely(temp_wkb.geometry) #transform geometry to be able to make output        
            to_geofeather(temp_wkb, os.path.join(aggr_change_path , '{}_{}_rel_change_2020_2100.feather'.format(country, ssp)), crs="EPSG:4326") #save as geofeather

C:\Users\snn490\AppData\Local\Continuum\miniconda3\envs\py38\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
pop:   0%|                                                                                    | 0/6865 [00:00<?, ?it/s]

aggregate population data for FRA scenario SSP1


pop:  80%|██████████████████████████████████████████████████████████▏              | 5467/6865 [26:27<06:33,  3.55it/s]

In [6]:
# soft clip of cisi for countries

#set pathways
cisi_path = os.path.abspath(r'\\labsdfs.labs.vu.nl\labsdfs\BETA-IVM-BAZIS\snn490\Outputs\Exposure\Final_cisi_files\CISI\010_degree')
country_shapes_path = os.path.abspath(r'C:\Users\snn490\surfdrive\Datasets\Administrative_boundaries\global_countries_buffer')

#import data
#cisi_index = pd.read_feather(cisi_path)
cisi_globe_pd = from_geofeather(os.path.join(cisi_path, 'CISI_global.feather')) 
shape_countries = from_geofeather(os.path.join(country_shapes_path,'global_countries_advanced.geofeather'))


In [7]:
for country in country_lst:
    country_shape = shape_countries[shape_countries['ISO_3digit'] == country]
    if country_shape.empty == False: #if ISO_3digit in shape_countries
        spat_tree = pygeos.STRtree(cisi_globe_pd.geometry)
        cisi_country = (cisi_globe_pd.loc[spat_tree.query(country_shape.geometry.iloc[0],predicate='intersects').tolist()]).sort_index(ascending=True) #get grids that overlap with cover_box
        cisi_country = cisi_country.reset_index(drop=True) #get index as column and name column grid_number
        
        # create new path
        aggr_change_path = os.path.abspath(os.path.join(r'\\labsdfs.labs.vu.nl\labsdfs\BETA-IVM-BAZIS\snn490\Datasets\Population_data', 'per_country_aggregated_change')) 
        Path(aggr_change_path).mkdir(parents=True, exist_ok=True) #create new directory, if it doesn't exist yet

        #save as geofeather
        to_geofeather(cisi_country, os.path.join(aggr_change_path , '{}_{}_rel_change_2020_2100.feather'.format(country, ssp)), crs="EPSG:4326") #save as geofeather
        #save as gpkg
        temp_df = functions.transform_to_gpd((cisi_country)) #transform df to gpd with shapely geometries
        temp_df.to_file(os.path.join(aggr_change_path , '{}_{}_rel_change_2020_2100.gpkg'.format(country, ssp)), layer=' ', driver="GPKG")


In [81]:
cisi_country

,CISI,Subscore_energy,Subscore_transportation,Subscore_water,Subscore_waste,Subscore_telecommunication,Subscore_healthcare,Subscore_education,geometry
0,0.000127,0.000002,0.000125,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((-5.2 48.6, -5.1 48.6, -5.1 48.5, -5...."
1,0.000141,0.000004,0.000137,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((-5.2 48.5, -5.1 48.5, -5.1 48.4, -5...."
2,0.006298,0.000017,0.001091,0.000057,0.000002,0.005131,0.000000,0.000000,"POLYGON ((-5.1 48.6, -5 48.6, -5 48.5, -5.1 48..."
3,0.000567,0.000005,0.000356,0.000000,0.000027,0.000000,0.000180,0.000000,"POLYGON ((-5.1 48.5, -5 48.5, -5 48.4, -5.1 48..."
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((-4.9 48.5, -4.8 48.5, -4.8 48.4, -4...."
...,...,...,...,...,...,...,...,...,...
6860,0.031363,0.021196,0.004812,0.000013,0.000000,0.002565,0.000991,0.001787,"POLYGON ((9.5 42.5, 9.6 42.5, 9.6 42.4, 9.5 42..."
6861,0.026037,0.020126,0.004720,0.000009,0.000000,0.000838,0.000180,0.000163,"POLYGON ((9.5 42.4, 9.6 42.4, 9.6 42.3, 9.5 42..."
6862,0.006320,0.000000,0.003309,0.000000,0.000280,0.002565,0.000096,0.000070,"POLYGON ((9.5 42.3, 9.6 42.3, 9.6 42.2, 9.5 42..."
6863,0.010111,0.000000,0.002088,0.000000,0.000000,0.007696,0.000084,0.000243,"POLYGON ((9.5 42.2, 9.6 42.2, 9.6 42.1, 9.5 42..."


In [8]:
# aggregate population at cisi-level, calculate changes

def value_zonal_stat(x,world_grid):
    try:
        return zonal_stats(x,world_grid,stats="sum")
    except:
        return [{'sum': 0}]

for ssp in ssp_lst:
        # load data
        #pop_data_2020 = rxr.open_rasterio(os.path.join(pop_data_clip_path,'{}_{}_2020.tif'.format(country, ssp)),masked=True).squeeze()
        #pop_data_2100 = rxr.open_rasterio(os.path.join(pop_data_clip_path,'{}_{}_2100.tif'.format(country, ssp)),masked=True).squeeze()
        
        pop_data_2020 = os.path.join(pop_data_clip_path,'{}_{}_2020.tif'.format(country, ssp))
        #pop_data_2100 = os.path.join(pop_data_clip_path,'{}_{}_2100.tif'.format(country, ssp))
        
        #world_pop = os.path.abspath(os.path.join(r'\\labsdfs.labs.vu.nl\labsdfs\BETA-IVM-BAZIS\data_catalogue\open_street_map\worldpop', 'ppp_2020_1km_Aggregated.tif'))
        
        
        temp_df = functions.transform_to_gpd((cisi_country)) #transform df to gpd with shapely geometries
        
        #calculate tot_pop_2020 per CISI grid cell
        tqdm.pandas(desc='pop')
        temp_df['tot_pop_2020'] = temp_df.geometry.progress_apply(lambda x: value_zonal_stat(x,pop_data_2020))
        temp_df['tot_pop_2020'] = temp_df['tot_pop_2020'].apply(lambda x: x[0]['sum']) 
        
        #calculate tot_pop_2100 per CISI grid cell
        tqdm.pandas(desc='pop')
        temp_df['tot_pop_2100'] = temp_df.geometry.progress_apply(lambda x: value_zonal_stat(x,pop_data_2100))
        temp_df['tot_pop_2100'] = temp_df['tot_pop_2100'].apply(lambda x: x[0]['sum']) 
        
        temp_df['rel_change_pop'] = (temp_df["tot_pop_2100"] - temp_df["tot_pop_2020"])/ temp_df["tot_pop_2020"] * 100
        
        #save as gpkg
        temp_df.to_file(os.path.join(aggr_change_path , '{}_{}_rel_change_2020_2100.gpkg'.format(country, ssp)), layer=' ', driver="GPKG")
        
        #save as geofeather
        temp_wkb = pd.DataFrame(temp_df) 
        temp_wkb['geometry'] = pygeos.from_shapely(temp_wkb.geometry) #transform geometry to be able to make output        
        to_geofeather(temp_wkb, os.path.join(aggr_change_path , '{}_{}_rel_change_2020_2100.feather'.format(country, ssp)), crs="EPSG:4326") #save as geofeather
        


C:\Users\snn490\AppData\Local\Continuum\miniconda3\envs\py38\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
pop: 100%|█████████████████████████████████████████████████████████████████████████| 6865/6865 [34:05<00:00,  3.36it/s]


In [9]:
temp_df

,CISI,Subscore_energy,Subscore_transportation,Subscore_water,Subscore_waste,Subscore_telecommunication,Subscore_healthcare,Subscore_education,geometry,tot_pop_2020
0,0.000127,0.000002,0.000125,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((-5.20000 48.55833, -5.10000 48.55833...",44.0
1,0.000141,0.000004,0.000137,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((-5.20000 48.45833, -5.10000 48.45833...",60.0
2,0.006298,0.000017,0.001091,0.000057,0.000002,0.005131,0.000000,0.000000,"POLYGON ((-5.10000 48.55833, -5.00000 48.55833...",191.0
3,0.000567,0.000005,0.000356,0.000000,0.000027,0.000000,0.000180,0.000000,"POLYGON ((-5.10000 48.45833, -5.00000 48.45833...",92.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((-4.90000 48.45833, -4.80000 48.45833...",0.0
...,...,...,...,...,...,...,...,...,...,...
6860,0.031363,0.021196,0.004812,0.000013,0.000000,0.002565,0.000991,0.001787,"POLYGON ((9.50000 42.45833, 9.60000 42.45833, ...",5033.0
6861,0.026037,0.020126,0.004720,0.000009,0.000000,0.000838,0.000180,0.000163,"POLYGON ((9.50000 42.35833, 9.60000 42.35833, ...",2676.0
6862,0.006320,0.000000,0.003309,0.000000,0.000280,0.002565,0.000096,0.000070,"POLYGON ((9.50000 42.25833, 9.60000 42.25833, ...",316.0
6863,0.010111,0.000000,0.002088,0.000000,0.000000,0.007696,0.000084,0.000243,"POLYGON ((9.50000 42.15833, 9.60000 42.15833, ...",337.0


In [91]:
temp_df['tot_pop_2020'][temp_df['tot_pop_2020'] > 0]



Series([], Name: tot_pop_2020, dtype: int64)

In [89]:
temp_df

,CISI,Subscore_energy,Subscore_transportation,Subscore_water,Subscore_waste,Subscore_telecommunication,Subscore_healthcare,Subscore_education,geometry,tot_pop_2020
0,0.000127,0.000002,0.000125,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((-5.20000 48.55833, -5.10000 48.55833...",0
1,0.000141,0.000004,0.000137,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((-5.20000 48.45833, -5.10000 48.45833...",0
2,0.006298,0.000017,0.001091,0.000057,0.000002,0.005131,0.000000,0.000000,"POLYGON ((-5.10000 48.55833, -5.00000 48.55833...",0
3,0.000567,0.000005,0.000356,0.000000,0.000027,0.000000,0.000180,0.000000,"POLYGON ((-5.10000 48.45833, -5.00000 48.45833...",0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((-4.90000 48.45833, -4.80000 48.45833...",0
...,...,...,...,...,...,...,...,...,...,...
6860,0.031363,0.021196,0.004812,0.000013,0.000000,0.002565,0.000991,0.001787,"POLYGON ((9.50000 42.45833, 9.60000 42.45833, ...",0
6861,0.026037,0.020126,0.004720,0.000009,0.000000,0.000838,0.000180,0.000163,"POLYGON ((9.50000 42.35833, 9.60000 42.35833, ...",0
6862,0.006320,0.000000,0.003309,0.000000,0.000280,0.002565,0.000096,0.000070,"POLYGON ((9.50000 42.25833, 9.60000 42.25833, ...",0
6863,0.010111,0.000000,0.002088,0.000000,0.000000,0.007696,0.000084,0.000243,"POLYGON ((9.50000 42.15833, 9.60000 42.15833, ...",0


In [4]:
#create lists
country_lst = ['FRA','DEU','ITA']
year_lst = ['2020', '2100']
ssp_lst = ['SSP1', 'SSP3', 'SSP5']
####################################################################################
   # aggregate population at cisi-level, calculate changes, and project on CISI #
####################################################################################

def value_zonal_stat(x,world_grid):
    try:
        return zonal_stats(x,world_grid,stats="sum")
    except:
        return [{'sum': 0}]

for country in country_lst:
    print(country)

    #aggregate data
    for ssp in ssp_lst:
        print(ssp)

FRA
SSP1
SSP3
SSP5
DEU
SSP1
SSP3
SSP5
ITA
SSP1
SSP3
SSP5
